In [5]:
import numpy as np
from datasets import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.ensemble import IsolationForest

from incidents_dataset import load_compressed_images

In [6]:
def batch_generator(batch_size):
    generator = load_compressed_images('dataset/cached_train.bin')
    
    buffer = []
    labels = []
    for image, label in generator:
        if len(buffer) < batch_size:
            buffer.append(image.flatten())
            labels.append(label.flatten())
        else:
            yield np.stack(buffer), np.stack(labels)
            buffer.clear()
            labels.clear()

In [ ]:
# 2. Incremental PCA 설정
n_components = 2  # 축소할 차원 수
ipca = IncrementalPCA(n_components=n_components)

batch_size = 256

# 3. Generator로부터 데이터를 배치 단위로 읽어와 Incremental PCA 학습
generator = batch_generator(batch_size)
print('batch size:', batch_size)
# partial_fit: Incremental PCA의 주성분을 점진적으로 학습
for (batch, labels) in tqdm(generator, ncols=75):
    if batch.shape[0] > 1:  # 배치 크기가 1보다 큰 경우만 처리
        ipca.partial_fit(batch)
        
# 4. 학습된 Incremental PCA를 사용하여 데이터 차원 축소
# 다시 generator를 사용하여 변환
transformed_data = []

generator = batch_generator(batch_size)
for (batch, labels) in tqdm(generator, ncols=75):
    if batch.shape[0] > 1:  # 배치 크기가 1보다 큰 경우만 처리
        transformed_batch = ipca.transform(batch)
        transformed_data.append(transformed_batch)
        
final_data = np.vstack(transformed_data)  # 모든 배치를 모아서 최종 결과 생성
print(f"Reduced data shape: {final_data.shape}")

0it [00:00, ?it/s]

In [7]:
# final_data는 차원이 축소된 데이터라고 가정
# n_components = 2로 설정하여 시각화 용도로 다시 축소
# pca_vis = PCA(n_components=2)
# data_2d = pca_vis.fit_transform(final_data)

# Isolation Forest를 사용하여 이상치 탐지
iso_forest = IsolationForest(contamination=0.05, random_state=42)  # contamination: 이상치 비율
outlier_labels = iso_forest.fit_predict(final_data)

# 정상 데이터와 이상치를 분리
normal_data = final_data[outlier_labels == 1]
outlier_data = final_data[outlier_labels == -1]

# 시각화
plt.figure(figsize=(12, 8))
plt.scatter(normal_data[:, 0], normal_data[:, 1], c='blue', label='Normal')
plt.scatter(outlier_data[:, 0], outlier_data[:, 1], c='red', label='Outliers')
plt.title('PCA Visualization of Reduced Data with Outliers')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

# 이상치 제거 후의 데이터
cleaned_data = final_data[outlier_labels == 1]
print(f"Original shape: {final_data.shape}")
print(f"Cleaned data shape: {cleaned_data.shape}")

NameError: name 'final_data' is not defined